In [2]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [6]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [7]:
result = model(tokens)

In [8]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [9]:
int(torch.argmax(result.logits))+1

4

In [24]:
r = requests.get('https://www.dineout.co.in/chennai/arab-street-ashok-nagar-west-chennai-23276/review')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*more.*')
results = soup.find_all('span', {'class':regex})
reviews = [result.text for result in results]

In [26]:
reviews

['very good',
 "Could have been better.. Mayonaisse taste bland.. I don't like it",
 'Customer service could be improved.',
 'The manager was rude... Threw our card n bill n dint show slightest respect',
 'Quite slow service',
 'Food is good. But they have restricted in some items when I booked through Dine out which is not good when we are visiting in a group of 9-10 people.',
 'not good.',
 "wrost one please don't  make the same mistake",
 "one of the best Restaurant in Chennai. Such an Awesome place. Food was Great and delicious. \nEspecially Mr.Malik was so kind and his service was the Best. I've never seen a waiter explaining so much about the dishes. People out there Must Try Arabstreet!!!!!????????",
 'Good taste, Service is good,, Thanks to Malik -, Supervisor']

In [27]:
import numpy as np
import pandas as pd

In [28]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [29]:
results

[<span class="more">very good</span>,
 <span class="more">Could have been better.. Mayonaisse taste bland.. I don't like it</span>,
 <span class="more">Customer service could be improved.</span>,
 <span class="more">The manager was rude... Threw our card n bill n dint show slightest respect</span>,
 <span class="more">Quite slow service</span>,
 <span class="more">Food is good. But they have restricted in some items when I booked through Dine out which is not good when we are visiting in a group of 9-10 people.</span>,
 <span class="more">not good.</span>,
 <span class="more">wrost one please don't  make the same mistake</span>,
 <span class="more">one of the best Restaurant in Chennai. Such an Awesome place. Food was Great and delicious. 
 Especially Mr.Malik was so kind and his service was the Best. I've never seen a waiter explaining so much about the dishes. People out there Must Try Arabstreet!!!!!????????</span>,
 <span class="more">Good taste, Service is good,, Thanks to Malik -

In [30]:
results[0].text

'very good'

In [31]:
df['review'].iloc[0]

'very good'

In [32]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [33]:
sentiment_score(df['review'].iloc[1])

2

In [34]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [35]:
df

,review,sentiment
0,very good,5
1,Could have been better.. Mayonaisse taste blan...,2
2,Customer service could be improved.,3
3,The manager was rude... Threw our card n bill ...,1
4,Quite slow service,2
5,Food is good. But they have restricted in some...,3
6,not good.,1
7,wrost one please don't make the same mistake,1
8,one of the best Restaurant in Chennai. Such an...,5
9,"Good taste, Service is good,, Thanks to Malik ...",4


In [36]:
df['review'].iloc[3]

'The manager was rude... Threw our card n bill n dint show slightest respect'